In [13]:
# import statements 
import arcpy
import os
import pandas as pd

# allow for overwrite in case errors are made 
arcpy.env.overwriteOutput = True 

# setting workspace 
arcpy.env.workspace = r"R:\GEOG491_16440_FALL2025\Student_Data\gshaffer\FinalProj"

# Define workspace and geodatabase name
workspace = r"R:\GEOG491_16440_FALL2025\Student_Data\gshaffer\FinalProj"
gdb_name = "final_proj.gdb"
gdb_path = os.path.join(workspace, gdb_name)

# Check if the geodatabase exists, and delete it if so
if arcpy.Exists(gdb_path):
    arcpy.management.Delete(gdb_path)

# Create the file geodatabase
arcpy.management.CreateFileGDB(workspace, gdb_name)
print(f"Geodatabase {gdb_name} created successfully.")



Geodatabase final_proj.gdb created successfully.


In [21]:
# bringing over datasets and databases 
unesco_layer = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\sites_package.lpkx"
unesco = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\unesco"

arcpy.management.CopyFeatures(
    in_features=unesco_layer,
    out_feature_class=unesco
)

ccvi_layer = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\ccvi\\climate_conflict_points_layer.lpkx"

output_ccvi = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\ccvi"

arcpy.management.CopyFeatures(
    in_features=ccvi_layer,
    out_feature_class=output_ccvi
)

ccvi = output_ccvi

arcpy.ExcelToTable_conversion(
    Input_Excel_File= "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\Conservation\\Conservation.xlsx",
    Output_Table="R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\conservation_data",
    Sheet= "data"
)

conservation_data = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\conservation_data"

In [22]:
# checking the unesco data 
fieldlist_unesco = arcpy.ListFields(unesco) 
for field in fieldlist_unesco: 
    print(field.name + " " + field.type)

OBJECTID OID
Shape Geometry
dossier_id Integer
dossier String
property_id Integer
property_serial SmallInteger
property_transboundary SmallInteger
table_source String
element_serial_number String
element_id String
element_name_en String
element_name_fr String
element_state String
element_id_state String
element_iso2 String
element_longitude Double
element_latitude Double
element_CoreArea Double
element_BufferArea Double
element_precision_publish String
property_rev_bis String
property_name_en String
property_name_fr String
property_short_description_en String
property_short_description_fr String
property_inscribed Integer
property_inscribed_sec_dates String
property_danger SmallInteger
property_danger_date_end Integer
property_danger_list String
property_core_area Double
property_criteria_txt String
property_category String
property_states_name_en String
property_states_name_fr String
property_iso2 String
property_iso3 String
C1 SmallInteger
C2 SmallInteger
C3 SmallInteger
C4 SmallInte

In [4]:
# checking the conservation data 
fieldlist_conservation = arcpy.ListFields(conservation_data) 
for field in fieldlist_conservation: 
    print(field.name + " " + field.type)

OBJECTID OID
Row Double
id_soc Double
id_site Double
Properties String
States_Parties String
Years Double
Themes String
Threats String
Danger_List String
Category String
Document_Source String


In [29]:
# renaming id and changing id type to have a common column name and type in the unesco and conservation data 
arcpy.management.AlterField(
    in_table=conservation_data,
    field="id_site",
    new_field_name="property_id"
)


<Result 'R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\conservation_data'>

In [30]:
# print fieldnames again to check if it changed 
# checking the conservation data 
fieldlist_conservation = arcpy.ListFields(conservation_data) 
for field in fieldlist_conservation: 
    print(field.name + " " + field.type)

OBJECTID OID
Row Double
id_soc Double
property_id Double
Properties String
States_Parties String
Years Double
Themes String
Threats String
Danger_List String
Category String
Document_Source String


In [31]:
# THIS IS WHERE YOU CAN CHANGE THE SQL EXPRESSION TO FILTER BY DIFFERENT COUNTRIES (OR OTHER FIELDS/AREAS)S
# query by country + return list of UNESCO Sites
#read unesco world heritage site data and select sites by country
delim_field = arcpy.AddFieldDelimiters(unesco, "element_state")

# establishing the SQL expression that will be used in the for loop
sql_exp = delim_field + " = 'Egypt'"


In [32]:
# creating a search cursor that will return a list of sites and types for which the SQL expression is true
cursor = arcpy.da.SearchCursor(unesco, ["property_name_en", "property_category"], sql_exp)
# for loop that will iterate through all of the UNESCO sites from that country with their type listed
for row in cursor:
    print(f"Name = {row[0]}, Type = {row[1]}")
    found = True
# following query by country, print statement in case country does not have any listed UNESCO Heritage Sites
if not found:
    print("This country has no listed UNESCO World Heritage Sites")


Name = Abu Mena, Type = Cultural
Name = Saint Catherine Area, Type = Cultural
Name = Wadi Al-Hitan (Whale Valley), Type = Natural
Name = Memphis and its Necropolis – the Pyramid Fields from Giza to Dahshur, Type = Cultural
Name = Memphis and its Necropolis – the Pyramid Fields from Giza to Dahshur, Type = Cultural
Name = Ancient Thebes with its Necropolis, Type = Cultural
Name = Ancient Thebes with its Necropolis, Type = Cultural
Name = Ancient Thebes with its Necropolis, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbel to Philae, Type = Cultural
Name = Nubian Monuments from Abu Simbe

In [33]:
# create new column for the 2025 conservation data
arcpy.management.AddField(
 in_table= unesco,
 field_name="Review_25",
 field_type="TEXT"
)
delimfield_2025review = arcpy.AddFieldDelimiters(unesco, "Review_25")

# Load your Excel table
df_2025 = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\conservation_data"

In [34]:
# Build a set of site IDs that have a 2025 review
review_ids = set()

with arcpy.da.SearchCursor(df_2025, ["property_id"]) as s_cursor:
    for row in s_cursor:
        val = row[0]
        if val is not None:
            review_ids.add(int(val))   # converting the double into an integer


# Update UNESCO layer field Review_25 based on matching IDs
with arcpy.da.UpdateCursor(unesco, ["property_id", "Review_25"]) as cursor:
    for row in cursor:
        val = row[0]
        if val is not None:
            pid = int(val)  # integer to integer 
            
            if pid in review_ids:
                row[1] = "Yes"
                cursor.updateRow(row)


In [38]:
# creating a search cursor that will return a list of sites and types for which the SQL expression is true AND there are sites that are listed for review in 2025 
cursor = arcpy.da.SearchCursor(unesco, ["property_name_en", "Review_25"], sql_exp)
# for loop that will iterate through all of the UNESCO sites from that country with their type listed
for row in cursor:
    # Check if the "Review_25" column is "Yes"
    if row[1] == "Yes":  # row[1] is "Review_25"
        print(f"Name = {row[0]}, 2025 review = {row[1]}")
        found = True 
# print statement in case country does not have any listed UNESCO Heritage Sites
if not found:
    print("This country has no listed UNESCO World Heritage Sites")


Name = Abu Mena, 2025 review = Yes
Name = Saint Catherine Area, 2025 review = Yes
Name = Memphis and its Necropolis – the Pyramid Fields from Giza to Dahshur, 2025 review = Yes
Name = Memphis and its Necropolis – the Pyramid Fields from Giza to Dahshur, 2025 review = Yes
Name = Ancient Thebes with its Necropolis, 2025 review = Yes
Name = Ancient Thebes with its Necropolis, 2025 review = Yes
Name = Ancient Thebes with its Necropolis, 2025 review = Yes


In [39]:
# calculate average unesco world heritage site size (there is not a polygon dataset available) 
fc = unesco 
field = "element_CoreArea" 

values = [row[0] for row in arcpy.da.SearchCursor(fc, field, sql_exp)] 
mean_area = sum(values) / len(values) 

print("Mean area =", mean_area, "hecatares")

# area is in hectares for this data

Mean area = 4562.806086956522 hecatares


In [47]:
import pandas as pd
from arcgis.features import FeatureLayer


In [50]:
# Create a feature layer for the selected country at the beginning of analysis to reduce runtime and only spatially join data by the country 
arcpy.management.MakeFeatureLayer(unesco, "country_lyr")

arcpy.management.SelectLayerByAttribute("country_lyr", "NEW_SELECTION", sql_exp)


<Result 'country_lyr'>

In [63]:
ccvi_fc = "R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\final_proj.gdb\\ccvi"

#checking ccvi_fc field names before they are used in the next code block 
for f in arcpy.ListFields(ccvi_fc):
    print(f.name, f.type)


OBJECTID OID
Shape Geometry
pgid Integer
lat Double
lon Double
iso3 String
CLI_longterm_temperature_anomal Double
CLI_current_floods_raw Double
CLI_current_drought_raw Double
CLI_accumulated_cyclones_raw Double
CLI_current_cyclones_raw Double
CLI_accumulated_drought_raw Double
CLI_longterm_relative_sea_level Double
CLI_longterm_precipitation_anom Double
CLI_accumulated_wildfires_raw Double
CLI_accumulated_heavy_precipita Double
CLI_current_heatwave_raw Double
CLI_accumulated_floods_raw Double
CLI_accumulated_heatwave_raw Double
CLI_current_heavy_precipitation Double
CLI_current_wildfires_raw Double
CLI_longterm_temperature_anom_1 Double
CLI_current_floods Double
CLI_current_drought Double
CLI_accumulated_cyclones Double
CLI_current_cyclones Double
CLI_accumulated_drought Double
CLI_longterm_relative_sea_lev_1 Double
CLI_longterm_precipitation_an_1 Double
CLI_accumulated_wildfires Double
CLI_accumulated_heavy_precipi_1 Double
CLI_current_heatwave Double
CLI_accumulated_floods Double
CLI

In [82]:
# for UNESCO and CCVI: spatial join with polygons (the UNESCO data) with point data
# Completing the field mappings (before doing the spatial join so there is somewhere for the data to go once I join them)
field_mappings = arcpy.FieldMappings()
field_mappings.addFieldMap(field_map)

# adding a new field using field mapping to get ccvi data into the new layer
field_map = arcpy.FieldMap()
# where the data is drawn from and the correct field name (checked with the above field names) from ccvi_fc 
field_map.addInputField(ccvi_fc, "CCVI")  
# the merge rule for making the new field 
field_map.mergeRule = "MEAN"

# Rename output field 
out_field = field_map.outputField
out_field.name = "CCVI_AVG"
field_map.outputField = out_field

name_field_map = arcpy.FieldMap()
name_field_map.addInputField("country_lyr", "property_name_en")  # Replace with the actual name field
field_mappings.addFieldMap(name_field_map)

review_field_map = arcpy.FieldMap()
review_field_map.addInputField("country_lyr", "Review_25")  # Replace with the actual name field
field_mappings.addFieldMap(review_field_map)


# Run SpatialJoin
arcpy.analysis.SpatialJoin(
    target_features="country_lyr",
    join_features=ccvi_fc,
    out_feature_class="unesco_ccvi_avg",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    match_option="WITHIN_A_DISTANCE",
# using 110,0000 meters to represent 110 km, which is 2x 55km, which is the distance that the ccvi index va
    search_radius="110000 Meters",
    field_mapping=field_mappings
)


<Result 'R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\unesco_ccvi_avg.shp'>

In [86]:
fc_ccvi_avg = "unesco_ccvi_avg"

delim_field_review = arcpy.AddFieldDelimiters(fc_ccvi_avg, "Review_25")

# establishing the SQL expression that will be used in the for loop
sql_review = delim_field_review + " = 'Yes'"



In [104]:
fields = arcpy.ListFields(fc_ccvi_avg)

for field in fields:
    print(f"Field Name: {field.name}")


Field Name: FID
Field Name: Shape
Field Name: Join_Count
Field Name: TARGET_FID
Field Name: CCVI_AVG
Field Name: property_n
Field Name: Review_25


In [109]:
cursor = arcpy.da.SearchCursor(fc_ccvi_avg, ["property_n", "Review_25", "CCVI_AVG"], sql_review)
# for loop that will iterate through the selected country's sites that are up for 2025 review and their CCVI average
for row in cursor:
    # Check if the "Review_25" column is "Yes"
    if row[1] == "Yes":  # row[1] is "Review_25"
        print(f"Name = {row[0]}, 2025 review = {row[1]}, CCVI average = {row[2]}")
        found = True 
# print statement in case country does not have any listed UNESCO Heritage Sites
if not found:
    print("No sites up for review in 2025")


<class 'RuntimeError'>: A column was specified that does not exist.

In [98]:
# finding the average ccvi for the sites that are labeled as up for 2025 review to find our threshold to flag other sites 
fc_ccvi_avg = "unesco_ccvi_avg"
field_ccvi = "CCVI_AVG" 


numbers = [row[0] for row in arcpy.da.SearchCursor(fc_ccvi_avg, field_ccvi, sql_review)] 
mean_review_ccvi = sum(numbers) / len(numbers) 

print("Mean CCVI = {:.3f}".format(mean_review_ccvi))  # Rounded to 3 decimal places



Mean CCVI = 0.319


In [101]:
# SQL using mean CCVI to select new sites that should be marked for review in 2025 based on the average CCVI of the sites currently up for review
delim_field_ccvi = arcpy.AddFieldDelimiters(fc_ccvi_avg, "Review_25")
ccvi_sql = delim_field_review + " >= mean_review_ccvi"


In [107]:
cursor = arcpy.da.SearchCursor(fc_ccvi_avg, ["property_n", "Review_25", "CCVI_AVG"], ccvi_sql)
# for loop that will iterate through the selected country's sites that are up for 2025 review and their CCVI average
for row in cursor:
    # Check if the "Review_25" column is "Yes"
    if row[1] == "Yes":  # row[1] is "Review_25"
        print(f"Name = {row[0]}, 2025 review = {row[1]}, CCVI average = {row[2]}")
        found = True 
# print statement in case country does not have any listed UNESCO Heritage Sites
if not found:
    print("No sites up for review in 2025")


<class 'RuntimeError'>: A column was specified that does not exist.

In [77]:
# Create a feature layer for the selected country at the beginning of analysis to reduce runtime and only spatially join data by the country 
arcpy.management.MakeFeatureLayer"unesco_ccvi_avg", "New_Review")

arcpy.management.SelectLayerByAttribute("New_Review", "NEW_SELECTION", ccvi_sql)




<Result 'R:\\GEOG491_16440_FALL2025\\Student_Data\\gshaffer\\FinalProj\\unesco_ccvi_avg.shp'>